# Kalman filter PIT sensitivity

# Purpose
Check the PIT for various Kalman filters

# Methodology
* Filter a model test differently...
* PIT on each
* compare result

# Setup

In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.plot import track_plot
from vessel_manoeuvring_models.equation import Equation

## Load models:
# (Uncomment these for faster loading):
import vessel_manoeuvring_models.models.vmm_abkowitz  as vmm_abkowitz 

In [ ]:
from vessel_manoeuvring_models.data import kalman_filter
from vessel_manoeuvring_models.data.lowpass_filter import lowpass_filter
import scipy.stats as stats

In [ ]:
id=22774
df, units, meta_data = mdl.load(id=id)

In [ ]:
fig,ax=plt.subplots()

psi = df['psi']
t = df.index.total_seconds()
psi1d = np.gradient(psi,t)

psi_ = df['psi'].resample('0.2S').mean()
t_ = psi_.index.total_seconds()
psi1d_ = np.gradient(psi_,t_)
ax.plot(t, psi1d)
ax.plot(t_, psi1d_)


In [ ]:
df_filtered = pd.DataFrame()

for observation_covariance in [10,1000,10000,100000, 1000000]:
    
    df_ = kalman_filter.filter_and_transform(df=df, observation_covariance=observation_covariance)
    #df_ = kalman_filter.yaw(df=df, observation_covariance=observation_covariance)

    df_['observation_covariance'] = observation_covariance
    df_['t'] = df_.index
    df_filtered = df_filtered.append(df_, ignore_index=True)
    

In [ ]:
df_epsilons = pd.DataFrame()

for observation_covariance, group in df_filtered.groupby(by='observation_covariance'):
    
    df_epsilon_ = pd.DataFrame()
    
    for key in ['x0','y0','psi']:
        df_epsilon_[f'{key}'] = group[f'{key}_filtered'].values - df[key]
    
    
    df_epsilon_['t'] = df.index.total_seconds()
    df_epsilon_['observation_covariance'] = observation_covariance
    
    df_epsilons = df_epsilons.append(df_epsilon_, ignore_index=True)
    

In [ ]:
#
#for key in ['x0','y0','psi']:
#    fig,ax=plt.subplots()
#    fig.set_size_inches(15,5)
#    for observation_covariance, df_epsilon in df_epsilons.groupby(by='observation_covariance'):
#        df_epsilon.plot(x='t', y=key, alpha=0.1, label=observation_covariance, ax=ax)
#        
#for key in ['x0','y0','psi']:
#    fig,axes=plt.subplots(ncols=len(df_epsilons['observation_covariance'].unique()))
#    for ax,(observation_covariance, df_epsilon) in zip(axes,df_epsilons.groupby(by='observation_covariance')):
#        sns.displot(df_epsilon, x=key)
#
#        fig,ax2=plt.subplots()
#        stats.probplot(df_epsilon[key], dist="norm", plot=ax2);

In [ ]:
df_ = df_filtered.groupby(by='observation_covariance').get_group(10).copy()
df_.index = df_['t']
df_['t'] = df_.index.total_seconds()

In [ ]:

psi_ = df_['psi'].resample('0.50S').mean()
t_ = psi_.index.total_seconds()
psi1d_ = np.gradient(psi_,t_)

fig,ax=plt.subplots()
fig.set_size_inches(15,10)

df_.plot(x='t', y='psi1d_gradient', ax=ax)
df_.plot(x='t', y='psi1d', ax=ax)

ax.plot(t_, psi1d_, label='resample')
ax.legend()

In [ ]:

psi_ = df_['x0'].resample('1.0S').mean()
t_ = psi_.index.total_seconds()
psi1d_ = np.gradient(psi_,t_)

fig,ax=plt.subplots()
fig.set_size_inches(15,10)

df_.plot(x='t', y='x01d_gradient', ax=ax)
df_.plot(x='t', y='x01d', ax=ax)

ax.plot(t_, psi1d_, label='resample')
ax.legend()

In [ ]:
psi_ = df_['y0'].resample('0.50S').mean()
t_ = psi_.index.total_seconds()
psi1d_ = np.gradient(psi_,t_)

fig,ax=plt.subplots()
fig.set_size_inches(15,10)

#df_.plot(x='t', y='y01d_gradient', ax=ax)
df_.plot(x='t', y='y01d', ax=ax)

ax.plot(t_, psi1d_, label='resample')
ax.legend()

In [ ]:
df_ = df_filtered.groupby(by='observation_covariance').get_group(1000).copy()
df_.set_index('t', inplace=True)
df_['t']=df_.index.total_seconds()
ts = df_['t'].diff().mean()
fs = 1/ts

In [ ]:
fs

In [ ]:

psi_filt = lowpass_filter(data=df_['psi'], cutoff=10, fs=fs, order=5)

fig,ax=plt.subplots()
df_.plot(x='t',y='psi', ax=ax,  label='kalman')
ax.plot(df_['t'], psi_filt, label='lowpass')
ax.legend()

r_filt = np.gradient(psi_filt,df_['t'])
r_filt = lowpass_filter(data=r_filt, cutoff=1, fs=fs, order=5)

r1d_filt = np.gradient(r_filt,df_['t'])
r1d_filt = lowpass_filter(data=r1d_filt, cutoff=1, fs=fs, order=5)

fig,ax=plt.subplots()
df_.plot(x='t',y='r', ax=ax, label='kalman')
ax.plot(df_['t'], r_filt, label='lowpass')
ax.legend()

fig,ax=plt.subplots()
df_.plot(x='t',y='r1d', ax=ax, label='kalman')
ax.plot(df_['t'], r1d_filt, alpha=0.5, label='lowpass')
ax.legend()


In [ ]:
np.gradient(df_gradient['x0'], df_gradient.index)

In [ ]:
df_gradient['x0']

In [ ]:
def lowpass_gradient(df_,cutoff=1, order=5):
    df_gradient = df_[['x0','y0','psi']].copy()
    df_gradient.index=df_['t']
    df_gradient['x01d'] = np.gradient(df_gradient['x0'], df_gradient.index)
    df_gradient['y01d'] = np.gradient(df_gradient['y0'], df_gradient.index)
    df_gradient['r'] = np.gradient(df_gradient['psi'], df_gradient.index)
    
    df_gradient['x02d'] = np.gradient(df_gradient['x01d'], df_gradient.index)
    df_gradient['y02d'] = np.gradient(df_gradient['y01d'], df_gradient.index)
    df_gradient['r1d'] = np.gradient(df_gradient['r'], df_gradient.index)
    
    df_lowpass = pd.DataFrame(columns=df_gradient.columns, index=df_gradient.index)
    for key,value in df_gradient.items():
        df_lowpass[key] = lowpass_filter(data=value, cutoff=cutoff, fs=fs, order=order)
    
    return df_lowpass

In [ ]:
def df_filter(df_gradient, cutoff=1, order=5):
    
    df_lowpass = pd.DataFrame(columns=df_gradient.columns, index=df_gradient.index)
    for key,value in df_gradient.items():
        df_lowpass[key] = lowpass_filter(data=value, cutoff=cutoff, fs=fs, order=order)
        
    return df_lowpass

def lowpass(df_,cutoff=1, order=5):
    
    df_gradient = df_[['x0','y0','psi']].copy()
    df_gradient.index=df_['t']
    
    df_gradient[['x0','y0','psi']] = df_filter(df_gradient[['x0','y0','psi']], cutoff=cutoff, order=order)
    df_gradient['x01d'] = np.gradient(df_gradient['x0'], df_gradient.index)
    df_gradient['y01d'] = np.gradient(df_gradient['y0'], df_gradient.index)
    df_gradient['r'] = np.gradient(df_gradient['psi'], df_gradient.index)
    
    df_gradient[['x01d','y01d','r']] = df_filter(df_gradient[['x01d','y01d','r']], cutoff=cutoff, order=order)
    df_gradient['x02d'] = np.gradient(df_gradient['x01d'], df_gradient.index)
    df_gradient['y02d'] = np.gradient(df_gradient['y01d'], df_gradient.index)
    df_gradient['r1d'] = np.gradient(df_gradient['r'], df_gradient.index)
    
    df_gradient[['x02d','y02d','r1d']] = df_filter(df_gradient[['x02d','y02d','r1d']], cutoff=cutoff, order=order)
    
    
    return df_gradient

In [ ]:
df_lowpass_gradient = lowpass_gradient(df_=df_, cutoff=0.5, order=1)
df_lowpass = lowpass(df_=df_, cutoff=1, order=1)

states = [
     ['x0','x01d','x02d'],
     ['y0','y01d','y02d'],
     ['psi','r','r1d']
]

for dof in states:
    fig,axes=plt.subplots(ncols=3)
    fig.set_size_inches(15,4)
    for state,ax in zip(dof,axes):
        ax.set_title(state)
        #df_lowpass_gradient.plot(y=state, ax=ax, label='lowpass gradient')
        df_lowpass.plot(y=state, ax=ax, label='lowpass', alpha=0.5)
        df_.plot(x='t', y=state, ax=ax, label='kalman', alpha=0.5)
        
        
        


In [ ]:
df_lowpass = pd.DataFrame(lowpass_filter(data=df_, cutoff=0.01, fs=fs, order=5),
                          columns=df_.columns, index=df_.index)
df_lowpass['t'] = df_lowpass.index.total_seconds()
df_lowpass['r'] = np.gradient(df_lowpass['psi'],df_lowpass['t'])
df_lowpass['r1d'] = np.gradient(df_lowpass['r'],df_lowpass['t'])


In [ ]:

fig,ax=plt.subplots()
fig.set_size_inches(15,10)

df_.plot(x='t', y='r1d', label='kalman', ax=ax)
df_lowpass.plot(x='t', y='r1d', label='lowpass', ax=ax)

for resample in [0.1,0.2,0.3,0.5,1]:

    psi_ = df_['psi'].resample(f'{resample}S').mean()
    t_ = psi_.index.total_seconds()
    r_ = np.gradient(psi_,t_)
    r1d_ = np.gradient(r_,t_)
    ax.plot(t_, r1d_, label=resample)
ax.legend()